In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from IPython.display import Markdown, display

In [2]:
import os
import openai

# needed to synthesize responses later
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
# load documents
# documents = SimpleDirectoryReader("/home/surya/NEU/CS5100 FAI/Project/ResearchLens/experiments/RAG/pdfs").load_data()
documents = SimpleDirectoryReader("/home/surya/NEU/CS5100 FAI/Project/ResearchLens/uploads").load_data()

In [4]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from llama_index.llms.cohere import Cohere

cohere_model = Cohere(api_key="vORtxj32na8zl2ceIbxH1c5tNziAVWDdAy2x3sbX")

In [ ]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.text_splitter = text_splitter
Settings.llm = cohere_model

Re-using index from RAG_doc notebook

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model, transformations=[text_splitter]
)

In [ ]:
# storage_context = StorageContext.from_defaults(
#     persist_dir="./storage"
# )
# index = load_index_from_storage(storage_context=storage_context)

## Test chat

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    llm=cohere_model,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as explaining research papers."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True,
)

In [ ]:
# chat_engine = index.as_chat_engine(
#     chat_mode="condense_plus_context",
#     context_prompt=(
#         "You are a chatbot, able to have normal interactions, as well as talk"
#         " about an essay discussing Paul Grahams life."
#         "Here are the relevant documents for the context:\n"
#         "{context_str}"
#         "\nInstruction: Based on the above documents, provide a detailed answer for the user question below."
#     ),
# )

In [ ]:
response = chat_engine.chat("""
On single-turn dialogue, we evaluate the different methods on the subset of the test split of the
Anthropic HH dataset [1] with one step of human-assistant interaction. GPT-4 evaluations use the
preferred completions on the test as the reference to compute the win rate for different methods. As
there is no standard SFT model for this task, we start with a pre-trained Pythia-2.8B, use Preferred-FT
to train a reference model on the chosen completions such that completions are within distribution
of the model, and then train using DPO. We also compare against the best of 128 Preferred-FT
completions (we found the Best of N baseline plateaus at 128 completions for this task; see Appendix
Figure 4) and a 2-shot prompted version of the Pythia-2.8B base model, finding DPO performs as
well or better for the best-performing temperatures for each method.

What dataset was used to evaluate the RLHF Model?       
""".strip())

In [ ]:
# print(response.source_nodes[0].text)

In [ ]:
# for chat in chat_engine.chat_history:
#     print(chat.role)
#     print(chat.content)
#     print()

In [ ]:
response.source_nodes

In [ ]:
print(response)